In [30]:
import pandas as pd
import numpy as np
import sklearn

In [31]:
# read train,val and test from files

In [32]:
train = pd.read_csv('input/train.tsv', sep='\t')
val = pd.read_csv('input/val.tsv', sep='\t')
test = pd.read_csv('input/test.tsv', sep='\t')

In [33]:
X_train = train.iloc[:,1:152]

In [34]:
y_train = train.loc[:,"target"]

In [35]:
X_test = val.iloc[:,1:152]

In [36]:
y_test = val.loc[:,"target"]

Lasso : First solution

In [10]:
# from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
# from sklearn import model_selection
# from sklearn.metrics import mean_squared_error
# %matplotlib inline
# import matplotlib.pyplot as plt

In [58]:
# lasso = Lasso(max_iter=10000, normalize=True)

# lassocv = LassoCV(alphas=None, cv=10, max_iter=100000, normalize=True)
# lassocv.fit(X_train, y_train)
# lasso.set_params(alpha=lassocv.alpha_)
# print("Alpha=", lassocv.alpha_)
# lasso.fit(X_train, y_train)
# print("mse = ",mean_squared_error(y_test, lasso.predict(X_test)))
# print("best model coefficients:")
# pd.Series(lasso.coef_, index=X_train.columns)

Alpha= 4.432332110388478e-07
mse =  0.10985454883417317
best model coefficients:


graph_dim_1     -1.603218e-02
graph_dim_2      1.019848e-02
graph_dim_3     -4.062988e-03
graph_dim_4     -7.143024e-03
graph_dim_5      3.792590e-02
                     ...     
graph_dim_147    3.704284e-02
graph_dim_148   -5.730044e-03
graph_dim_149   -3.864395e-02
graph_dim_150    1.398705e-02
user_id          1.546575e-19
Length: 151, dtype: float64

Lasso : second solution

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

In [60]:
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',Lasso())
])

In [61]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.01,5,0.01)},
                      cv = 5, scoring="neg_mean_squared_error",verbose=3
                      )

In [ ]:
search.fit(X_train,y_train)

In [ ]:
search.best_params_

In [19]:
# get coefficients from Lasso regression

In [41]:
coefficients = search.best_estimator_.named_steps['model'].coef_

In [43]:
coefficients

array([-0.       ,  0.       ,  0.       , -0.       ,  0.       ,
       -0.       ,  0.       , -0.       , -0.       , -0.       ,
       -0.       , -0.       , -0.       ,  0.       , -0.       ,
       -0.       , -0.       ,  0.       , -0.       ,  0.       ,
       -0.       ,  0.       , -0.       ,  0.       ,  0.       ,
        0.       , -0.       ,  0.       , -0.       ,  0.       ,
        0.       , -0.       ,  0.       , -0.       , -0.       ,
        0.       , -0.       ,  0.       , -0.       , -0.       ,
        0.       , -0.       ,  0.       , -0.       ,  0.       ,
        0.       , -0.       , -0.       , -0.       ,  0.       ,
       -0.       ,  0.       ,  0.       ,  0.       , -0.       ,
       -0.       , -0.       , -0.       ,  0.       , -0.       ,
        0.       , -0.       , -0.       , -0.       ,  0.       ,
       -0.       , -0.       , -0.       ,  0.       ,  0.       ,
       -0.       ,  0.       ,  0.       ,  0.       , -0.    

In [27]:
# The importance of a feature is the absolute value of its coefficient

In [44]:
importance = np.abs(coefficients)

In [29]:
importance

array([0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 